In [ ]:
import os
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options

INPUT_FOLDER = "NameLists"  # 待查询名单的.xlsx文件所在的文件夹

# 初始化浏览器
options = Options()
options.add_argument("--start-maximized")
driver = webdriver.Edge(service=Service(), options=options) # 此处我使用的是edge浏览器，可自行更改

# 自动进入登录界面，需手动登录
driver.get("https://www.tianyancha.com/login")
input("Login page loaded. Please log in manually, then press Enter to continue...")

# 进入批量导出页面
driver.get("https://www.tianyancha.com/batch")
print("Navigated to batch export page.")

DIMENSION = "供应商" # 选择本次导出的维度，和导出页面表格里的选项一致

try:
    for file in os.listdir(INPUT_FOLDER):
        if file.endswith(".xlsx"):
            filepath = os.path.abspath(os.path.join(INPUT_FOLDER, file))
            print(f"Processing file: {file}")

            # 等待上传按钮出现
            upload_input = WebDriverWait(driver, 30).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "input[type='file']"))
            )
            print("Upload input located.")

            driver.execute_script("arguments[0].style.display = 'block';", upload_input)

            # 上传名单
            upload_input.send_keys(filepath)
            print("File uploaded successfully.")

            # 等待匹配完成
            print("Waiting for match...")
            time.sleep(12)

            # 自动点击 “更多维度导出” 按钮
            try:
                buttons = WebDriverWait(driver, 30).until(
                    # 此为网页元素“更多维度导出”按钮（靠右的按钮）的索引，可以按目的自行更改
                    # 若想点击“基础工商信息”（靠左的按钮），可以自行更改网页元素
                    EC.presence_of_all_elements_located((By.CLASS_NAME, "index_button__h8Hrs"))
                )
                
                if len(buttons) > 1:
                    buttons[1].click()
                    print("'More dimensions export' button clicked.")
                else:
                    raise Exception("Not enough buttons found with class name 'index_button__h8Hrs'")
                
            except Exception as e:
                print(f"Failed to click 'More dimensions export' button: {e}")
                driver.quit()
                exit()

            time.sleep(3) # 等待页面加载

            # 选择维度
            try:
                # 等待维度按钮出现
                dimension_button = WebDriverWait(driver, 30).until(
                    EC.presence_of_element_located((By.XPATH, f"//button[span[text()='{DIMENSION}']]"))
                )
                dimension_button.click()
                print(f"'{DIMENSION}' option selected.")
                
            except Exception as e:
                print(f"Failed to select '{DIMENSION}' option: {e}")
                driver.quit()
                exit()

            time.sleep(3)

            # 定位 “导出” 按钮
            export_options = WebDriverWait(driver, 30).until(
                EC.presence_of_element_located((By.CLASS_NAME, "index_CheckedBox__9snSk"))
            )
            print("Export options located.")

            # 找到并取消第一个复选框（csv），因为网页默认勾选了第一种文件格式。可自由更改。
            csv_checkbox = export_options.find_element(By.CLASS_NAME, "index_csv__qJgyu")
            if "fill=\"#0084FF\"" in csv_checkbox.get_attribute("innerHTML"):
                csv_checkbox.click()
                print("CSV export option unchecked.")

            # 找到并勾选第二个复选框（xls），此处因为我自己需要导出xls格式的文件。可自由更改。
            xls_checkbox = export_options.find_element(By.CLASS_NAME, "index_xls__1LAIH")
            if "fill=\"#0084FF\"" not in xls_checkbox.get_attribute("innerHTML"):
                xls_checkbox.click()
                print("XLS export option selected.")

            # 等待导出按钮出现
            time.sleep(2)

            # 点击导出按钮
            export_button = WebDriverWait(driver, 30).until(
                EC.presence_of_element_located((By.CLASS_NAME, "_50ab4.index_exportButton__9Jnq2._52bf6"))
            )
            export_button.click()
            print("Export initiated.")

            # 等待文件下载
            print("Waiting for file download...")
            time.sleep(5)

            # 返回批量导出页面
            print("Returning to batch export page...")
            driver.get("https://www.tianyancha.com/batch")
            print("Navigated back to batch export page.")

            # 等待页面加载
            time.sleep(3)

            # 打印完成信息
            print(f"Export task for {file} completed!")

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    driver.quit()
    print("Script execution completed!")

Navigated to batch export page.
Processing file: namelist_00.xlsx
Upload input located.
File uploaded successfully.
Waiting for match...
'More dimensions export' button clicked.
'裁判文书' option selected.
Export options located.
CSV export option unchecked.
XLS export option selected.
Export initiated.
Waiting for file download...
Returning to batch export page...
Navigated back to batch export page.
Export task for namelist_00.xlsx completed!
Processing file: namelist_01.xlsx
Upload input located.
File uploaded successfully.
Waiting for match...
'More dimensions export' button clicked.
'裁判文书' option selected.
Export options located.
CSV export option unchecked.
XLS export option selected.
Export initiated.
Waiting for file download...
Returning to batch export page...
Navigated back to batch export page.
Export task for namelist_01.xlsx completed!
Processing file: namelist_02.xlsx
Upload input located.
File uploaded successfully.
Waiting for match...
'More dimensions export' button click